# Analysis of Closest Hospitals to Nursing Homes in Ireland
- article done on [nursing homes and populations](https://hrbopenresearch.org/articles/3-65) based on need and availability
- inspired by [The relationship between distance to hospital and patient mortality in emergencies: an observational study](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2464671/), a UK study on distances from hospitals and mortality rates
- data comes from [DATA.GOV.IE](https://data.gov.ie/)
- list of [irish nursing homes](https://data.gov.ie/dataset/list-of-nursing-homes-in-ireland) and list of [irish hospitals](https://data.gov.ie/dataset/list-of-hospitals-in-ireland) from 2017
- using conda environment named 'data_analysis' (Python 3.12.3)

In [1]:
# import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pyproj
import math
from collections import Counter
import mpld3
from mpld3 import plugins

In [2]:
# data paths
DATA_GOV_PATH = '../data_gov_ie/'

In [3]:
df_dict = {}
for item in os.listdir(DATA_GOV_PATH):
    if '.csv' in item:
        df_dict[item.replace('.csv', '')] = pd.read_csv(DATA_GOV_PATH + item)

df_dict.keys()

dict_keys(['personsInCommunalEstablishments', 'disability', 'publicPrivateExpenditureFairDeals', 'irish_hospitals', 'placeOfDeathSPCCommunity', 'deaths_by_age', 'lengthOfStayInHospice', 'irish_nursing_homes', 'deaths_by_location'])

## Geographical Spread of Irish Nursing Homes

In [4]:
# check for missing values
print((df_dict['irish_nursing_homes'].isnull()).sum())
df_dict['irish_hospitals'] = df_dict['irish_hospitals'].drop(['eircode'], axis = 1)
print((df_dict['irish_hospitals'].isnull()).sum())

_id             0
Service name    0
Address         0
County          0
ig_xcord        0
ig_ycord        0
dtype: int64
_id        0
name       0
address    0
x          0
y          0
dtype: int64


- no missing values found for nursing home data
- dropped eircode column in hospital dataset due to 4 missing values

In [5]:
homes_df = df_dict['irish_nursing_homes'].drop_duplicates(subset = ['Service name', 'Address'])
hospitals_df = df_dict['irish_hospitals'].drop_duplicates(subset = ['name', 'address'])

In [6]:
%matplotlib inline

# create map of Ireland
fig, ax = plt.subplots(figsize=(8, 6))
basemap = Basemap(projection='merc', llcrnrlat=51, urcrnrlat=56, llcrnrlon=-11, urcrnrlon=-5, resolution='i', ax=ax)

# draw map features
basemap.drawcoastlines()
basemap.drawcountries()
basemap.fillcontinents(color='lightgray', zorder=0)

# convert ITM coordinates to geographic latitude and longitude coordinates
itm_proj = pyproj.Proj('epsg:29902')  
home_lon, home_lat = itm_proj(homes_df['ig_xcord'].values, homes_df['ig_ycord'].values, inverse=True)
hospital_lon, hospital_lat = itm_proj(hospitals_df['x'].values, hospitals_df['y'].values, inverse=True)

# plot map points
home_x, home_y = basemap(home_lon, home_lat)
hospital_x, hospital_y = basemap(hospital_lon, hospital_lat)
home_scatter = basemap.scatter(home_x, home_y, marker='o', color='red', edgecolors='black', s=15, zorder=3, label='Nursing Homes')
hospital_scatter = basemap.scatter(hospital_x, hospital_y, marker='o', color='blue', edgecolors='black', s=15, zorder=3, label='Hospitals')

# hover functionality
home_labels = [f'<div style="background-color: white; color: black; border: 1px solid black; padding: 5px;">{name}</div>' for name in homes_df['Service name']]
hospital_labels = [f'<div style="background-color: white; color: black; border: 1px solid black; padding: 5px;">{name}</div>' for name in hospitals_df['name']]

tooltip_home = plugins.PointHTMLTooltip(home_scatter, labels=home_labels)
tooltip_hospital = plugins.PointHTMLTooltip(hospital_scatter, labels=hospital_labels)

plugins.connect(fig, tooltip_home, tooltip_hospital)

# add title
plt.title('Locations of Hospitals and Nursing Homes in Ireland')
plt.legend()
# save plot
plt.savefig('../graphs/locations_homes_hospitals.png', bbox_inches='tight')
# show plot
mpld3.display()
# for zooming
# mpld3.show()


# save plot as HTML file
html_str = mpld3.fig_to_html(fig)
with open("../html_pages/hospitalsAndNursingHomes.html", "w") as f:
    f.write(html_str)

In [7]:
# create function to calculate distance between two points
def haversine_distance(home_lon, hospital_lon, home_lat, hospital_lat):
    """
    calculate the great circular distance between two points on earth specified in decimal degrees
    accepts input in irish grid and outputs distance in km
    """
    # convert ITM coordinates to geographic latitude and longitude coordinates
    itm_proj = pyproj.Proj('epsg:29902')
    lon1, lat1 = itm_proj(home_lon, home_lat, inverse=True)
    lon2, lat2 = itm_proj(hospital_lon, hospital_lat, inverse=True)
    
    # convert decimal degrees to radians
    lon1, lon2, lat1, lat2 = map(math.radians, [lon1, lon2, lat1, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # radius of earth in km is 6371
    km = 6371 * c
    return km

def straight_line_distance(lon1, lon2, lat1, lat2):
    """ 
    Calculate Euclidian distance. 
    """
    # convert ITM coordinates to geographic latitude and longitude coordinates
    itm_proj = pyproj.Proj('epsg:29902')
    lon1, lat1 = itm_proj(lon1, lat1, inverse=True)
    lon2, lat2 = itm_proj(lon2, lat2, inverse=True)

    # convert latitude and longitude from degrees to radians
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

    # difference in latitude and longitude
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # calculate distance using Euclidean formula
    distance = math.sqrt(delta_lat ** 2 + delta_lon ** 2)

    # radius of the Earth (in kilometers)
    # for more accurate distances, you can use the WGS84 ellipsoid model of the Earth
    # the radius of the Earth varies slightly depending on the latitude
    earth_radius = 6371.0

    # convert distance from radians to kilometers
    distance_km = earth_radius * distance

    return distance_km

In [8]:
all_dist = {}
for i in range(len(homes_df)):
    temp_dist = 1000000000000000000000000000000000
    temp_hospital = ""
    temp_address = ""
    temp_x = 0
    temp_y = 0
    for j in range(len(hospitals_df)):
        new_dist = straight_line_distance(homes_df['ig_xcord'][i], hospitals_df['x'][j], homes_df['ig_ycord'][i], hospitals_df['y'][j])
        if new_dist < temp_dist:
            temp_dist = new_dist
            temp_hospital = hospitals_df['name'][j]
            temp_x = hospitals_df['x'][j]
            temp_y = hospitals_df['y'][j]
            temp_address = hospitals_df['address'][j]
    all_dist[i] = {'home_name': homes_df['Service name'][i], 'home_address': homes_df['Address'][i] , 'home_x': homes_df['ig_xcord'][i], 'home_y': homes_df['ig_ycord'][i] , 'hospital_name': temp_hospital, 'hospital_address': temp_address, 'hospital_x': temp_x, 'hospital_y': temp_y,'distance': temp_dist}

In [9]:
all_dist = pd.DataFrame(all_dist)
distances_df = all_dist.transpose()
print(Counter(distances_df['hospital_name']), Counter(distances_df['hospital_address']))

Counter({"St Columcille's Hospital": 25, 'University Hospital Kerry': 25, 'South Infirmary Victoria University Hospital': 24, 'Mayo General Hospital': 22, 'Beaumont Hospital': 22, 'South Tipperary General Hospital': 21, 'Sligo Regional Hospital': 20, 'Roscommon County Hospital': 20, 'Merlin Park University Hospital': 20, "St Luke's General Hospital Kilkenny": 20, 'Naas General Hospital': 20, 'Nenagh Hospital': 18, "St Vincent's University Hospital": 17, 'Letterkenny University Hospital': 17, 'Cork University Hospital': 17, "St John's Hospital": 17, 'Mater Misericordiae Hospital': 17, 'University Hospital Waterford': 17, 'Connolly Hospital Dublin': 17, 'Wexford General Hospital': 15, 'Mallow General Hospital': 15, 'Ennis Hospital': 14, "St Michael's Hospital Dun Laoghaire": 14, 'Portiuncula Hospital': 14, "St James's Hospital Dublin": 13, 'Midland Regional Hospital Mullingar': 12, 'Bantry General Hospital': 12, 'University Hospital Galway': 11, 'Cavan General Hospital': 11, 'Our Lady of

In [10]:
row_data_loc = distances_df.loc[distances_df['distance'] == max(distances_df['distance'])]
row_data_loc

,home_name,home_address,home_x,home_y,hospital_name,hospital_address,hospital_x,hospital_y,distance
440,St. Anne's Community Nursing Home,"St. Anne's Community Nursing Home, Westport Ro...",65628.63054,251117.3017,Mayo General Hospital,"Westport Road, Castlebar, Co. Mayo,",114195.8524,289970.125,89.5571


In [11]:
# create map of Ireland
fig, ax = plt.subplots(figsize=(8, 6))
basemap = Basemap(projection='merc', llcrnrlat=51, urcrnrlat=56, llcrnrlon=-11, urcrnrlon=-5, resolution='i', ax = ax)

# draw map features
basemap.drawcoastlines()
basemap.drawcountries()
basemap.fillcontinents(color='lightgray', zorder=0)

# convert ITM coordinates to geographic latitude and longitude coordinates
itm_proj = pyproj.Proj('epsg:29902')  
home_lon, home_lat = itm_proj(distances_df['home_x'].values, distances_df['home_y'].values, inverse=True)

# plot map points
home_x, home_y = basemap(home_lon, home_lat)
home_labels = [str(name) for name in homes_df['Service name']]
print(len(home_x), len(home_y), len(distances_df['distance'].values))

number_of_risky_homes = 0
red_x = []
red_y = []
red_name = []
blue_x = []
blue_y = []
blue_name = []
green_x = []
green_y = []
green_name = []
for lon, lat, size, name in zip(home_x, home_y, distances_df['distance'], home_labels):
    # distance categories defined by: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2464671/
    # 10km increase in straight-line distance is associated with a 1% absolute increase in mortality

    if size >= 21:
        red_x.append(lon)
        red_y.append(lat)
        red_name.append(name)
        number_of_risky_homes += 1
    elif size >= 11:
        blue_x.append(lon)
        blue_y.append(lat)
        blue_name.append(name)
    elif size < 11:
        green_x.append(lon)
        green_y.append(lat)
        green_name.append(name)
    else:
        print('anomaly')

red_scatter = basemap.scatter(red_x, red_y, marker='o', color='red', edgecolors='black', s=15, zorder=9, label='21km+')
blue_scatter = basemap.scatter(blue_x, blue_y, marker='o', color='blue', edgecolors='black', s=15, zorder=8,  label='11km-20km')
green_scatter = basemap.scatter(green_x, green_y, marker='o', color='green', edgecolors='black', s=15, zorder=7, label='0km-10km')

# hover functionality
red_labels = [f'<div style="background-color: white; color: black; border: 1px solid red; padding: 5px;">{name}</div>' for name in red_name]
blue_labels = [f'<div style="background-color: white; color: black; border: 1px solid blue; padding: 5px;">{name}</div>' for name in blue_name]
green_labels = [f'<div style="background-color: white; color: black; border: 1px solid green; padding: 5px;">{name}</div>' for name in green_name]


tooltip_red = plugins.PointHTMLTooltip(red_scatter, labels=red_labels)
tooltip_blue = plugins.PointHTMLTooltip(blue_scatter, labels=blue_labels)
tooltip_green = plugins.PointHTMLTooltip(green_scatter, labels=green_labels)


plugins.connect(fig, tooltip_red, tooltip_blue, tooltip_green)


# add title
plt.title('Distances from Nursing Homes to Hospitals')
plt.legend()
# save plot
plt.savefig('../graphs/map_greatest_distance_from_homes_to_hospitals.png', bbox_inches='tight')
# show plot
mpld3.display()
# for zooming
# mpld3.show()

# save plot as HTML file
html_str = mpld3.fig_to_html(fig)
with open("../html_pages/distancesFromHospitals.html", "w") as f:
    f.write(html_str)


578 578 578


In [12]:
print(number_of_risky_homes, 'out of', len(distances_df) ,'homes are at least 21km from the closest hospital. Therefore,', (str(number_of_risky_homes / len(distances_df) * 100) + '% of homes are potentially at risk.'))

240 out of 578 homes are at least 21km from the closest hospital. Therefore, 41.522491349480966% of homes are potentially at risk.


In [13]:
df_dict.keys()

dict_keys(['personsInCommunalEstablishments', 'disability', 'publicPrivateExpenditureFairDeals', 'irish_hospitals', 'placeOfDeathSPCCommunity', 'deaths_by_age', 'lengthOfStayInHospice', 'irish_nursing_homes', 'deaths_by_location'])

In [14]:
pc_death_place_df = df_dict['placeOfDeathSPCCommunity']
death_by_age_df = df_dict['deaths_by_age']
length_hospice_df = df_dict['lengthOfStayInHospice']
overall_deaths_by_location  = df_dict['deaths_by_location']


In [15]:
df_dict['deaths_by_location']

,Statistic Label,Quarter,Cause of Death,County of Residence of Deceased,UNIT,VALUE
0,Deaths Registered,2022Q1,Total deaths (A00-Y89),Ireland,Number,9535.0
1,Deaths Registered,2022Q1,Total deaths (A00-Y89),Cork City Council,Number,376.0
2,Deaths Registered,2022Q1,Total deaths (A00-Y89),Clare County Council,Number,236.0
3,Deaths Registered,2022Q1,Total deaths (A00-Y89),Cavan County Council,Number,150.0
4,Deaths Registered,2022Q1,Total deaths (A00-Y89),Cork County Council,Number,766.0
...,...,...,...,...,...,...
1339,Deaths Registered,2023Q3,All other causes,Tipperary County Council,Number,81.0
1340,Deaths Registered,2023Q3,All other causes,Waterford City & County Council,Number,68.0
1341,Deaths Registered,2023Q3,All other causes,Westmeath County Council,Number,41.0
1342,Deaths Registered,2023Q3,All other causes,Wicklow County Council,Number,53.0
